# Gradient boosting

**Gradient boosting** es una técnica genérica que interpreta FSAM como un algoritmo descenso por gradiente para un problema de minimización en un espacio funcional:
$$\hat{\boldsymbol{f}}%
=\operatorname*{argmin}_{\boldsymbol{f}}\mathcal{L}(\boldsymbol{f})
=\operatorname*{argmin}_{\boldsymbol{f}}\sum_{i=1}^N \ell(y_i,f(\boldsymbol{x}_i))$$
donde, por simplicidad, las funciones se representan por sus valores en el conjunto de entrenamiento, $\boldsymbol{f}=(f(\boldsymbol{x}_1),\dotsc,f(\boldsymbol{x}_N))^t$.

## Iteración $m$

En la iteración $m$ se desea construir un modelo $\boldsymbol{f}_m$ añadiendo una función base, $F_m$, a un modelo previo, $\boldsymbol{f}_{m-1}$, tal que $\mathcal{L}(\boldsymbol{f}_m)\leq\mathcal{L}(\boldsymbol{f}_{m-1})$. Visto como un problema de minimización en un espacio funcional, descenso por gradiente consiste en escoger la "dirección" de máximo descenso, esto es, la del negativo del gradiente de $\mathcal{L}(\boldsymbol{f})$ en $\boldsymbol{f}_{m-1}$, $\boldsymbol{g}_m$, y construir el nuevo modelo como:
$$\boldsymbol{f}_m=\boldsymbol{f}_{m-1}-\beta_m \boldsymbol{g}_m
\qquad\text{con}\qquad%
g_{im}=\left[\frac{\partial\ell(y_i,f(\boldsymbol{x}_i))}{\partial f(\boldsymbol{x}_i)}\right]_{f_{m-1}(\boldsymbol{x}_i)}$$
donde el factor de aprendizaje puede escogerse por búsqueda lineal. Ahora bien, el algoritmo no es muy útil si se limita a funciones representadas por sus valores en el conjunto de entrenamiento, ya que no podemos generalizar. Para poder hacerlo, gradient boosting ajusta el aprendiz débil al negativo del gradiente:
$$F_m=\operatorname*{argmin}_F\sum_{i=1}^N(-g_{im}-F(\boldsymbol{x}_i))^2$$

## Algoritmo básico

El algoritmo básico prescinde de $\beta_m$ pero incluye un **shrinkage factor** $0<\nu\leq 1$ para facilitar la regularización:
1. Inicializar $f_0(\boldsymbol{x})=\operatorname{argmin}_F\sum_{i=1}^N\ell(y_i,F(\boldsymbol{x}_i))$
2. **for** $\;m=1:M\;$ **do**
3. $\quad$ Calcular el negativo del gradiente o (pseudo-)residuo $r_{im}=-\left[\dfrac{\partial\ell(y_i,f(\boldsymbol{x}_i))}{\partial f(\boldsymbol{x}_i)}\right]_{f_{m-1}(\boldsymbol{x}_i)}$
4. $\quad$ Usar el aprendiz débil para hallar $F_m=\operatorname*{argmin}_F\sum_{i=1}^N(r_{im}-F(\boldsymbol{x}_i))^2$
5. $\quad$ Actualizar $f_m(\boldsymbol{x})=f_{m-1}(\boldsymbol{x})+\nu F_m(\boldsymbol{x})$
6. Devolver $f(\boldsymbol{x})=f_M(\boldsymbol{x})$

## Pérdidas y residuos usuales

En regresión, $y_i\in\mathbb{R}$ y se suele usar el error cuadrático como función de pérdida. Más concretamente, se usa la mitad del error cuadrático pues el negativo de su derivada es el residuo de la predicción:
$$\ell(y_i,f(\boldsymbol{x}_i))=\frac{1}{2}(y_i-f(\boldsymbol{x}_i))^2%
\qquad\text{con}\qquad%
r_{im}=y_i-f(\boldsymbol{x}_i)$$
Una alternativa al error cuadrático en regresión es el error absoluto. El negativo de su derivada es el signo del residuo de la predicción:
$$\ell(y_i,f(\boldsymbol{x}_i))=\lvert y_i-f(\boldsymbol{x}_i)\rvert%
\qquad\text{con}\qquad%
r_{im}=\operatorname{sgn}(y_i-f(\boldsymbol{x}_i))$$
En clasificación binaria, $\tilde{y}_i\in\{-1,+1\}$ y se suele usar la pérdida exponencial o la log-pérdida binaria:
$$\begin{align*}
&\ell(y_i,f(\boldsymbol{x}_i))=\exp(-\tilde{y}_i f(\boldsymbol{x}_i))%
&\qquad\text{con}\qquad%
&r_{im}=\tilde{y}_i\exp(-\tilde{y}_i f(\boldsymbol{x}_i))\\%
&\ell(y_i,f(\boldsymbol{x}_i))=\log(1+\exp(-\tilde{y}_i f(\boldsymbol{x}_i)))%
&\qquad\text{con}\qquad%
&r_{im}=\tilde{y}_i-\sigma(\tilde{y}_if(\boldsymbol{x}_i))%
\end{align*}$$
En clasificación multiclase, $y_i\in\{1,\dotsc,C\}$ y se usa la log-pérdida. El algoritmo básico se extiende para ajustar $C$ modelos aditivos, uno por cada clase, cuyas predicciones se normalizan mediante una softmax:
$$\begin{gather*}
\ell(y_i,\{f_c(\boldsymbol{x}_i)\}_{c=1}^C)=-\sum_c \mathbb{I}(y_i=c)\log \pi_{ic}\\%
\text{con}\qquad%
r_{im}=\mathbb{I}(y_i=c)-\pi_{ic}
\qquad\text{y}\qquad%
\pi_{ic}=S(f_1(\boldsymbol{x}_i),\dotsc,f_C(\boldsymbol{x}_i))_c
\end{gather*}$$